# Predictive Analytics Model for KServe

## Overview
This notebook trains and saves a **Predictive Analytics model** in KServe-compatible format for the `predictive-analytics` InferenceService.

**Model Type**: Time series forecasting with Random Forest  
**Purpose**: Predict future resource usage (CPU, memory, disk, network)  
**Deployment**: KServe InferenceService with sklearn runtime

## KServe Integration (Issue #13 Fix)

This notebook implements the fix for [Issue #13](https://github.com/tosin2013/openshift-aiops-platform/issues/13) where models were registering as `"model"` instead of `"predictive-analytics"`.

### Problem Solved
- **Before**: Models saved to `/mnt/models/cpu_usage_step_0_model.pkl` (flat structure)
- **After**: Models saved to `/mnt/models/predictive-analytics/model.pkl` (KServe structure)
- **Result**: Model registers correctly as `"predictive-analytics"` ✅

### Architecture
```
Notebook Training → /mnt/models/predictive-analytics/model.pkl
                    ↓
KServe InferenceService (storageUri: pvc://model-storage-pvc/predictive-analytics)
                    ↓
Model registered as: "predictive-analytics"
                    ↓
Endpoint: /v1/models/predictive-analytics:predict
```

## Prerequisites
- Model storage PVC mounted at `/mnt/models`
- Python environment with sklearn, pandas, numpy
- Access to `src/models/predictive_analytics.py` module

## What This Notebook Does
1. ✅ Imports the `PredictiveAnalytics` module
2. ✅ Generates synthetic time series training data
3. ✅ Trains multi-metric forecasting models (CPU, memory, disk, network)
4. ✅ Saves in KServe-compatible format: `/mnt/models/predictive-analytics/model.pkl`
5. ✅ Validates the model works correctly
6. ✅ Tests prediction endpoint format

## Setup Section

### Import Libraries and Configure Environment

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Setup path for src/models module - works from any directory
def find_models_path():
    """Find src/models path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent.parent / 'src' / 'models' if '__file__' in dir() else None,
        Path.cwd().parent.parent / 'src' / 'models',
        Path('/workspace/repo/src/models'),
        Path('/opt/app-root/src/openshift-aiops-platform/src/models'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'predictive_analytics.py').exists():
            return str(p)
    # Try relative path search
    current = Path.cwd()
    for _ in range(5):
        models_path = current / 'src' / 'models'
        if models_path.exists() and (models_path / 'predictive_analytics.py').exists():
            return str(models_path)
        current = current.parent
    return None

models_path = find_models_path()
if models_path:
    sys.path.insert(0, models_path)
    print(f"✅ Models path found: {models_path}")
else:
    print("⚠️ Models path not found - using fallback implementation")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import joblib
import warnings
warnings.filterwarnings('ignore')

# Import PredictiveAnalytics module
try:
    from predictive_analytics import PredictiveAnalytics, generate_sample_timeseries_data
    print("✅ PredictiveAnalytics module imported successfully")
    USING_MODULE = True
except ImportError as e:
    print(f"❌ Failed to import PredictiveAnalytics module: {e}")
    print("   Please ensure src/models/predictive_analytics.py is available")
    USING_MODULE = False

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("\n✅ All libraries imported successfully")

### Configure Model Storage Paths

In [ ]:
# Configure storage paths
# Use /mnt/models for persistent storage (model-storage-pvc)
# Fallback to local for development outside cluster
MODELS_DIR = Path('/mnt/models') if Path('/mnt/models').exists() else Path('/opt/app-root/src/models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Model name must match InferenceService name
MODEL_NAME = 'predictive-analytics'
MODEL_DIR = MODELS_DIR / MODEL_NAME  # Will be created by save_models()

print(f"📁 Model Storage Configuration:")
print(f"   Base directory: {MODELS_DIR}")
print(f"   Model name: {MODEL_NAME}")
print(f"   Expected KServe path: {MODEL_DIR}/model.pkl")
print(f"   PVC available: {'✅ Yes' if MODELS_DIR == Path('/mnt/models') else '⚠️ No (using local)'}")

if not USING_MODULE:
    print("\n❌ Cannot proceed without PredictiveAnalytics module")
    raise ImportError("PredictiveAnalytics module required for this notebook")

## Data Generation Section

### Generate Synthetic Time Series Data

In [ ]:
# Generate synthetic time series data for training
print("📊 Generating synthetic time series data...")
print("   This simulates realistic infrastructure metrics with patterns:")
print("   - Daily cycles (higher during business hours)")
print("   - Weekly patterns (weekday vs weekend)")
print("   - Trends (gradual growth over time)")
print("   - Noise (random variations)")

# Generate 2000 samples (about 7 days at 5-minute intervals)
sample_data = generate_sample_timeseries_data(n_samples=2000)

print(f"\n✅ Generated {len(sample_data)} samples")
print(f"   Columns: {', '.join(sample_data.columns)}")
print(f"   Shape: {sample_data.shape}")
print(f"   Date range: {sample_data['timestamp'].min()} to {sample_data['timestamp'].max()}")

# Display sample data
print("\n📋 Sample data (first 5 rows):")
display(sample_data.head())

# Show statistics
print("\n📈 Data Statistics:")
display(sample_data.describe())

### Visualize Training Data

In [ ]:
# Visualize the generated time series data
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
fig.suptitle('Synthetic Time Series Training Data', fontsize=16, fontweight='bold')

metrics = ['cpu_usage', 'memory_usage', 'disk_usage', 'network_in', 'network_out']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.plot(sample_data['timestamp'], sample_data[metric], color=color, alpha=0.7, linewidth=1)
    ax.set_title(f'{metric.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

# Hide the last subplot (we have 5 metrics in a 3x2 grid)
axes[2, 1].set_visible(False)

plt.tight_layout()
plt.show()

print("✅ Visualization complete")

### Split Data for Training and Validation

In [ ]:
# Split data: 80% training, 20% validation
split_point = int(len(sample_data) * 0.8)

train_data = sample_data.iloc[:split_point].copy()
val_data = sample_data.iloc[split_point:].copy()

print(f"📊 Data Split:")
print(f"   Training samples: {len(train_data)} ({len(train_data)/len(sample_data)*100:.1f}%)")
print(f"   Validation samples: {len(val_data)} ({len(val_data)/len(sample_data)*100:.1f}%)")
print(f"   Training period: {train_data['timestamp'].min()} to {train_data['timestamp'].max()}")
print(f"   Validation period: {val_data['timestamp'].min()} to {val_data['timestamp'].max()}")

## Model Training Section

### Initialize and Train PredictiveAnalytics Model

In [ ]:
# Initialize PredictiveAnalytics model
print("🔬 Initializing PredictiveAnalytics model...")

# Configure model parameters
FORECAST_HORIZON = 12  # Predict 12 time steps ahead
LOOKBACK_WINDOW = 24   # Use 24 historical time steps

predictor = PredictiveAnalytics(
    forecast_horizon=FORECAST_HORIZON,
    lookback_window=LOOKBACK_WINDOW
)

print(f"   Forecast horizon: {FORECAST_HORIZON} time steps")
print(f"   Lookback window: {LOOKBACK_WINDOW} time steps")
print(f"   Target metrics: {', '.join(predictor.target_metrics)}")

# Train the model
print(f"\n🎯 Training models on {len(train_data)} samples...")
print("   This will train separate models for each metric:")
print("   - CPU usage")
print("   - Memory usage")
print("   - Disk usage")
print("   - Network in")
print("   - Network out")

training_results = predictor.train(train_data)

print(f"\n✅ Training completed!")
print(f"   Models trained: {training_results['models_trained']}")
print(f"   Feature count: {training_results['feature_count']}")
print(f"   Forecast horizon: {training_results['forecast_horizon']}")
print(f"   Lookback window: {training_results['lookback_window']}")

### Evaluate Model Performance

In [ ]:
# Display detailed metrics for each model
print("📊 Model Performance Metrics:\n")
print("=" * 80)

for metric_name, results in training_results['metrics'].items():
    print(f"\n{metric_name.upper().replace('_', ' ')}:")
    print(f"  Mean Absolute Error (MAE):  {results['mae']:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {results['rmse']:.4f}")
    print(f"  R² Score: {results['r2']:.4f}")
    print(f"  Training samples: {results['training_samples']}")
    print(f"  Test samples: {results['test_samples']}")
    print("  " + "-" * 60)

print("\n" + "=" * 80)

# Calculate average R² across all metrics
avg_r2 = np.mean([r['r2'] for r in training_results['metrics'].values()])
print(f"\n📈 Average R² Score: {avg_r2:.4f}")

if avg_r2 > 0.8:
    print("✅ Excellent model performance!")
elif avg_r2 > 0.6:
    print("✅ Good model performance")
else:
    print("⚠️ Model performance could be improved - consider more training data")

## Model Validation Section

### Test Predictions on Validation Data

In [ ]:
# Make predictions on validation data
print("🔮 Making predictions on validation data...")

predictions = predictor.predict(val_data.head(50))

print(f"\n✅ Predictions generated:")
print(f"   Timestamp: {predictions['timestamp']}")
print(f"   Metrics predicted: {len(predictions['predictions'])}")
print(f"   Lookback window used: {predictions['lookback_window']}")

# Display predictions for each metric
print("\n📊 Prediction Results:\n")
for metric_name, pred_data in predictions['predictions'].items():
    forecast = pred_data['forecast']
    confidence = pred_data.get('confidence', [0.5] * len(forecast))
    
    print(f"{metric_name.upper().replace('_', ' ')}:")
    print(f"  Forecast values (first 5): {[f'{v:.2f}' for v in forecast[:5]]}")
    print(f"  Confidence (avg): {np.mean(confidence):.2%}")
    print()

### Visualize Predictions vs Actual

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Predictions vs Actual Values (Validation Set)', fontsize=16, fontweight='bold')

# Select metrics to visualize
vis_metrics = ['cpu_usage', 'memory_usage', 'disk_usage', 'network_in']

for idx, metric in enumerate(vis_metrics):
    ax = axes[idx // 2, idx % 2]
    
    # Plot actual values
    actual_vals = val_data[metric].head(50).values
    ax.plot(range(len(actual_vals)), actual_vals, label='Actual', color='blue', alpha=0.6, linewidth=2)
    
    # Plot predictions (if available)
    if metric in predictions['predictions']:
        forecast = predictions['predictions'][metric]['forecast']
        # Start prediction from lookback_window position
        pred_start = predictions['lookback_window']
        ax.plot(range(pred_start, pred_start + len(forecast)), 
               forecast, label='Predicted', color='red', alpha=0.6, linewidth=2, linestyle='--')
    
    ax.set_title(f'{metric.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Value')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualization complete")

## Model Saving Section

### Save Model in KServe-Compatible Format

This is the critical step that implements the **Issue #13 fix**!

In [ ]:
# Save model with KServe-compatible structure
print("💾 Saving model in KServe-compatible format...\n")
print("=" * 80)
print("KSERVE COMPATIBILITY (Issue #13 Fix)")
print("=" * 80)
print(f"\n📂 Directory Structure:")
print(f"   Base: {MODELS_DIR}")
print(f"   Model subdirectory: {MODEL_NAME}/")
print(f"   Full path: {MODELS_DIR}/{MODEL_NAME}/model.pkl")
print(f"\n✅ This matches KServe's expected structure!")
print(f"   storageUri: pvc://model-storage-pvc/{MODEL_NAME}")
print(f"   KServe loads: /mnt/models/{MODEL_NAME}/model.pkl")
print(f"   Registers as: '{MODEL_NAME}' (not 'model')")

# Save using the updated save_models() method
# kserve_compatible=True is the default
print(f"\n🔧 Calling save_models() with kserve_compatible=True...")
predictor.save_models(str(MODELS_DIR), kserve_compatible=True)

# Verify the saved model
expected_path = MODELS_DIR / MODEL_NAME / 'model.pkl'
print(f"\n🔍 Verifying saved model...")

if expected_path.exists():
    size_kb = expected_path.stat().st_size / 1024
    print(f"\n✅ SUCCESS! Model saved correctly:")
    print(f"   Location: {expected_path}")
    print(f"   Size: {size_kb:.2f} KB")
    print(f"   Structure: ✅ KServe-compatible subdirectory")
    
    # Check for migration artifacts
    old_files = list(MODELS_DIR.glob('*_step_*_model.pkl'))
    if old_files:
        print(f"\n⚠️ Found {len(old_files)} old model files (should be cleaned up)")
    else:
        print(f"   Cleanup: ✅ No old files found")
    
    print(f"\n📡 KServe Deployment Info:")
    print(f"   InferenceService name: {MODEL_NAME}")
    print(f"   Model registration: '{MODEL_NAME}' (fixed!)")
    print(f"   Endpoint: /v1/models/{MODEL_NAME}:predict")
    print(f"   Expected response to: curl http://<ip>:8080/v1/models")
    print(f"   {{\"models\":[\"{MODEL_NAME}\"]}}  ← Correct! ✅")
    print(f"   NOT: {{\"models\":[\"model\"]}}  ← This was the bug ❌")
    
else:
    print(f"\n❌ ERROR: Model not found at expected location: {expected_path}")
    print(f"   Check the save_models() implementation")

print("\n" + "=" * 80)

### Test Model Loading (Verification)

In [ ]:
# Verify the model can be loaded back
print("🧪 Testing model loading...")

# Create a new instance and load the saved model
test_predictor = PredictiveAnalytics()
test_predictor.load_models(str(MODELS_DIR))

print(f"\n✅ Model loaded successfully!")
print(f"   Models loaded: {len(test_predictor.models)}")
print(f"   Forecast horizon: {test_predictor.forecast_horizon}")
print(f"   Lookback window: {test_predictor.lookback_window}")
print(f"   Target metrics: {', '.join(test_predictor.target_metrics)}")

# Make a test prediction
test_predictions = test_predictor.predict(val_data.head(50))
print(f"\n✅ Test prediction successful!")
print(f"   Metrics predicted: {len(test_predictions['predictions'])}")

print(f"\n🎉 Model is ready for KServe deployment!")

## Deployment Verification Section

### Generate KServe Test Commands

In [ ]:
# Generate commands for testing the deployed model
print("📋 KServe Deployment Test Commands:\n")
print("=" * 80)
print("After deploying the InferenceService, run these commands to verify:\n")

print("# 1. Get the predictor pod IP")
print(f"PREDICTOR_IP=$(oc get pod -l serving.kserve.io/inferenceservice={MODEL_NAME} -o jsonpath='{{.items[0].status.podIP}}')")
print(f"echo \"Predictor IP: $PREDICTOR_IP\"\n")

print("# 2. List available models (should return 'predictive-analytics', not 'model')")
print("curl http://${PREDICTOR_IP}:8080/v1/models")
print(f"# Expected: {{\"models\":[\"{MODEL_NAME}\"]}}  ✅\n")

print("# 3. Check model status")
print(f"curl http://${{PREDICTOR_IP}}:8080/v1/models/{MODEL_NAME}")
print(f"# Expected: {{\"name\":\"{MODEL_NAME}\",\"ready\":true}}  ✅\n")

print("# 4. Test prediction endpoint")
print(f"curl -X POST http://${{PREDICTOR_IP}}:8080/v1/models/{MODEL_NAME}:predict \\")
print("  -H 'Content-Type: application/json' \\")
print("  -d '{\"instances\": [[0.5, 0.6, 0.4, 100, 80]]}'")
print("# Expected: Prediction response with forecast values  ✅\n")

print("=" * 80)
print("\n✅ If all commands work, Issue #13 is fixed!")

## Summary

### What Was Accomplished

✅ **Model trained** with multi-metric forecasting (CPU, memory, disk, network)  
✅ **Saved in KServe format**: `/mnt/models/predictive-analytics/model.pkl`  
✅ **Issue #13 fixed**: Model will register as `"predictive-analytics"` not `"model"`  
✅ **Validated**: Model can be loaded and makes predictions  
✅ **Ready for deployment**: Compatible with KServe InferenceService

### Next Steps

1. **Deploy InferenceService** (if not already deployed):
   ```yaml
   apiVersion: serving.kserve.io/v1beta1
   kind: InferenceService
   metadata:
     name: predictive-analytics
   spec:
     predictor:
       model:
         name: predictive-analytics
         runtime: sklearn-pvc-runtime
         storageUri: "pvc://model-storage-pvc/predictive-analytics"
   ```

2. **Verify deployment** using the commands above

3. **Test from coordination engine**:
   - Ensure coordination engine can call `/v1/models/predictive-analytics:predict`
   - Verify predictions work end-to-end

4. **Monitor predictions**:
   - Check prediction accuracy over time
   - Retrain periodically with new data

### References

- **Issue**: [#13 - KServe model registration fix](https://github.com/tosin2013/openshift-aiops-platform/issues/13)
- **Module**: `src/models/predictive_analytics.py`
- **Training script**: `src/models/train_predictive_analytics.py`
- **Documentation**: `src/models/KSERVE_FIX_README.md`